<a href="https://colab.research.google.com/github/ShreyG12345/DATA70202/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this code is sensitive and must not be shared publicly and can only be viewed by instructors.


# Raw Data

import pandas as pd

df = pd.read_csv(processed_data.csv)


# Basic stats
df.shape
df.info()
df.describe(include='all')
df.columns

display(df.describe())
display(df.describe(include='object'))

# Drop Ni Number
df.drop(columns=['ni_number'], inplace=True)

# Display Missing Variables
missing = df.isnull().sum().sort_values(ascending=False)
display(missing[missing > 0])

# AGES

df['date_of_birth_month'] = pd.to_datetime(df['date_of_birth_month'], errors='coerce')

# 6 April 2016 (New SPA rules)
reference_date = pd.to_datetime("2016-04-06")
df['age_at_new_SP'] = (reference_date - df['date_of_birth_month']).dt.days / 365
df['age_band_at_new_SP'] = pd.cut(
    df['age_at_new_SP'],
    bins=[0, 16, 18, 50, 60, 65, 200],
    labels=['Under 16', '16-17', '19-49', '50-59', '60-65', '66+'],
    right=False
)

# Drop records under 16 and over 60! at the reference date
initial_count = len(df)
df = df[(df['age_at_new_SP'] >= 16) & (df['age_at_new_SP'] <= 60)]
removed_count = initial_count - len(df)

print("Age band counts (after filtering to 16–60):")
print(df['age_band_at_new_SP'].value_counts())
print(f"\nRemoved {removed_count} records with age outside 16–60 range.")


# Service years
df['total_service_years'] = (
    df['pensionable_service_final_salary_scheme'].fillna(0) +
    df['active_service_on_full_time_basis_2015_scheme'].fillna(0)
)

df['total_service_years'] = pd.to_numeric(df['total_service_years'], errors='coerce')

df['service_band'] = pd.cut(
    df['total_service_years'],
    bins=[0, 10, 21, 31, 41, 46, df['total_service_years'].max() + 1],
    labels=['Under 10', '10–20', '21–30', '31–40', '41–45', 'Over 45'],
    right=False
)

print("Total Service Band Counts:")
print(df['service_band'].value_counts().sort_index())
over_45_service_count = (df['total_service_years'] > 45).sum()
print(f"\nMembers with total service over 45 years: {over_45_service_count}")

# Actual Pay

df['actual_pay_over_12_months_final_salary_scheme'] = pd.to_numeric(
    df['actual_pay_over_12_months_final_salary_scheme'], errors='coerce'
)

pay_bins = [0, 5000, 10000, 20000, 40000, 60000, 80000, 100000, 150000, 200000, 500000, 1_000_000, float('inf')]
pay_labels = [
    'Under £5k', '£5k–10k', '£10k–20k', '£20k–40k', '£40k–60k', '£60k–80k', '£80k–100k',
    '£100k–150k', '£150k–200k', '£200k–500k', '£500k–1m', 'Over £1m'
]

df['pay_band'] = pd.cut(
    df['actual_pay_over_12_months_final_salary_scheme'],
    bins=pay_bins,
    labels=pay_labels,
    right=False,
    include_lowest=True
)

print("Actual Pay Band Distribution:")
print(df['pay_band'].value_counts().sort_index())

max_pay = df['actual_pay_over_12_months_final_salary_scheme'].max()
min_pay = df['actual_pay_over_12_months_final_salary_scheme'].min()
over_1m_count = (df['actual_pay_over_12_months_final_salary_scheme'] > 1_000_000).sum()
zero_pay_count = (df['actual_pay_over_12_months_final_salary_scheme'] == 0).sum()


print(f"\nMembers with actual pay over £1 million: {over_1m_count}")
print(f"Members with actual pay of £0: {zero_pay_count}")
print(f"Max actual pay: £{max_pay:,.2f}")
print(f"Min actual pay: £{min_pay:,.2f}")

# 0 pay investigation
zero_pay_df = df[df['actual_pay_over_12_months_final_salary_scheme'] == 0]
print("Total members with £0 pay:", len(zero_pay_df))
zero_service_df = zero_pay_df[zero_pay_df['pensionable_service_final_salary_scheme'] == 0]
print("Members with £0 pay AND 0 pensionable service:", len(zero_service_df))
print("\nGender distribution (zero pay):")
print(zero_pay_df['gender'].value_counts(dropna=False))
print("\nCurrent scheme distribution (zero pay):")
print(zero_pay_df['current_scheme'].value_counts(dropna=False))
zero_pay_df['date_of_birth_month'] = pd.to_datetime(zero_pay_df['date_of_birth_month'], errors='coerce')
zero_pay_df['date_joined_scheme_month'] = pd.to_datetime(zero_pay_df['date_joined_scheme_month'], errors='coerce')
zero_pay_df['years_in_scheme'] = (pd.to_datetime("today") - zero_pay_df['date_joined_scheme_month']).dt.days / 365.25
print("\nYears in scheme (zero pay):")
print(zero_pay_df['years_in_scheme'].describe())
zero_pay_zero_service_under_1_year = zero_service_df[
    (pd.to_datetime("today") - pd.to_datetime(zero_service_df['date_joined_scheme_month'], errors='coerce')).dt.days / 365.25 < 1
]
print("\nMembers with £0 pay, 0 service, and < 1 year in scheme:", len(zero_pay_zero_service_under_1_year))

# Remove them

df = df[
    ~(
        (df['actual_pay_over_12_months_final_salary_scheme'] == 0) &
        (df['pensionable_service_final_salary_scheme'] == 0)
    )
]

# Part time
df['part_time_proportion'] = pd.to_numeric(df['part_time_proportion'], errors='coerce')

total_members = len(df)
non_missing = df['part_time_proportion'].notna().sum()
missing = df['part_time_proportion'].isna().sum()
zero_prop = (df['part_time_proportion'] == 0).sum()
non_zero = non_missing - zero_prop
over_100 = (df['part_time_proportion'] > 100).sum()
exact_100 = (df['part_time_proportion'] == 100).sum()
under_50 = (df['part_time_proportion'] < 50).sum()
between_50_100 = df[
    (df['part_time_proportion'] >= 50) &
    (df['part_time_proportion'] < 100)
].shape[0]

print(f"Members With valid part-time proportion: {non_missing} ({non_missing/total_members:.1%})")
print(f"Members missing part-time proportion: {missing} ({missing/total_members:.1%})")
print(f"\n0% part-time (full-time members): {zero_prop} ({zero_prop/total_members:.1%})")
print(f"Non-zero part-time members: {non_zero} ({non_zero/total_members:.1%})")
print(f"\nOver 100% part-time proportion: {over_100}")
print(f"Exactly 100%: {exact_100}")
print(f"Under 50%: {under_50}")
print(f"Between 50–99%: {between_50_100}")

# Drop rows with part_time_proportion > 1 (100%)
df = df[df['part_time_proportion'] <= 1]

# Protection status

df['protection_status'] = df['protection_status'].astype(str).str.upper().str.strip()
prot_u_or_n = df[df['protection_status'].isin(['U', 'N'])]
total_u_n = prot_u_or_n.shape[0]
missing_care = prot_u_or_n['accrued_2015_scheme_pension_at_valuation_date'].isna().sum()
missing_pct = (missing_care / total_u_n) * 100
total_missing_care = df['accrued_2015_scheme_pension_at_valuation_date'].isna().sum()
total_members = df.shape[0]

print(f"Total members with protection status 'U' or 'N': {total_u_n}")
print(f"Protected members missing post-2015 CARE pension: {missing_care}")
print(f"That’s {missing_pct:.2f}% of 'U'/'N' members")

fill_0_cols = [
    'dependant_service_final_salary',
    'x2015_rate_of_pensionable_pay_final_salary_scheme',
    'x2016_rate_of_pensionable_pay_final_salary_scheme',
    'x2017_rate_of_pensionable_pay_final_salary_scheme',
    'x2018_rate_of_pensionable_pay_final_salary_scheme',
    'x2019_rate_of_pensionable_pay_final_salary_scheme',
    'x2015_16_employee_contribution_history',
    'x2016_17_employee_contribution_history',
    'x2017_18_employee_contribution_history',
    'x2018_19_employee_contribution_history',
    'x2019_20_employee_contribution_history',
    'x2015_scheme_club_pension_subject_to_cpi_0_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_25_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_5_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_6_percent_increases',
    'x2015_scheme_club_pension_subject_to_earnings_increases',
    'actual_pensionable_earnings_over_year_police_2015_scheme',
    'annual_rate_of_pensionable_earnings_police_2015_scheme',
    'active_service_on_full_time_basis_2015_scheme',
    'service_on_full_time_basis_final_salary_scheme',
    'transferred_in_service_final_salary_scheme',
    'x2015_scheme_reckonable_service',
    'actual_pay_over_12_months_final_salary_scheme_orig',
    'pensionable_pay_final_salary_scheme_orig'
]

for col in fill_0_cols:
    if col in df.columns:
        df.loc[:, col] = df[col].fillna(0)

missing = df.isnull().sum().sort_values(ascending=False)
display(missing[missing > 0])

"""dates stay nan and rank is currently unknown although likely cat"""

# Rank

# Rank decode map
rank_map = {
    80: "Constable",
    81: "Sergeant",
    82: "Inspector",
    83: "Chief Inspector",
    84: "Superintendent",
    85: "Chief Superintendent",
    86: "Assistant Chief Constable / Commander",
    87: "Deputy Chief Constable / Assistant Commissioner",
    88: "Chief Constable / Commissioner",
    89: "Deputy Commissioner",
    90: "Commissioner",
    99: "Divorce Pension Credit Member"
}

# Apply mapping
df['rank_label'] = df['rank'].map(rank_map)
df['rank_label'] = df['rank_label'].fillna("Unknown")
df['rank_orig_label'] = df['rank_orig'].map(rank_map)
df['rank_orig_label'] = df['rank_orig'].fillna("Unknown")

df.columns

#******************************* Pre-Processing***************************

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

date_cols = [
    'date_of_birth_month',
    'date_joined_scheme_month',
    'date_of_transition_to_2015_scheme_month',
    'date_attains_state_pension_age_legislation_month'
]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Derive Time Features
df['age_at_join'] = (df['date_joined_scheme_month'] - df['date_of_birth_month']).dt.days / 365.25
df['years_until_state_pension'] = (df['date_attains_state_pension_age_legislation_month'] - pd.to_datetime('today')).dt.days / 365.25
df['years_in_scheme'] = (pd.to_datetime('today') - df['date_joined_scheme_month']).dt.days / 365.25

# Categorical Columns
categorical_cols = [
    'current_scheme',
    'previous_final_salary_scheme_membership_if_any',
    'gender',
    'marital_status',
    'part_time',
    'career_break',
    'protection_status',
    'public_sector_transfer_club_pstc_transfer_in_indicator',
    'previous_final_salary_scheme_orig',
    'adm_ons_name'  # will be replaced with 'county'
]

# Clean Categorical
def clean_categorical(col):
    return (
        col.astype(str)
           .str.strip()
           .str.upper()
           .replace({
               'FEMALE': 'F',
               'MALE': 'M',
               'YES': 'Y',
               'NO': 'N'
           })
    )

for col in categorical_cols:
    if col in df.columns:
        df[col] = clean_categorical(df[col])
    else:
        print(f"Skipping column not found: {col}")

#  County Codes
county_map = {
    'AVON AND SOMERSET': 'AVS', 'BEDFORDSHIRE': 'BED', 'CAMBRIDGESHIRE': 'CAM',
    'CHESHIRE': 'CHS', 'CLEVELAND': 'CLV', 'CUMBRIA': 'CUM', 'DERBYSHIRE': 'DER',
    'DEVON & CORNWALL': 'DEC', 'DORSET': 'DOS', 'DURHAM': 'DUR', 'DYFED-POWYS': 'POW',
    'ESSEX': 'ESS', 'GLOUCESTERSHIRE': 'GLO', 'GREATER MANCHESTER': 'GMN', 'GWENT': 'GWN',
    'HAMPSHIRE': 'HMP', 'HERTFORDSHIRE': 'HRT', 'HUMBERSIDE': 'HUM', 'KENT': 'KNT',
    'LANCASHIRE': 'LAN', 'LEICESTERSHIRE': 'LEI', 'LINCOLNSHIRE': 'LIN', 'LONDON, CITY OF': 'LDN',
    'MERSEYSIDE': 'MER', 'METROPOLITAN POLICE': 'MET', 'NCA': 'NCA', 'NORFOLK': 'NOR',
    'NORTH WALES': 'NWA', 'NORTH YORKSHIRE': 'NYK', 'NORTHAMPTONSHIRE': 'NTH',
    'NORTHUMBRIA': 'NMB', 'NOTTINGHAMSHIRE': 'NOT', 'SOUTH WALES': 'SWL',
    'SOUTH YORKSHIRE': 'SYK', 'STAFFORDSHIRE': 'STF', 'SUFFOLK': 'SFK', 'SURREY': 'SUR',
    'SUSSEX': 'SUS', 'THAMES VALLEY': 'THV', 'WARWICKSHIRE': 'WAR', 'WEST MERCIA': 'WMC',
    'WEST MIDLANDS': 'WMD', 'WEST YORKSHIRE': 'WYK', 'WILTSHIRE': 'WIL'
}

if 'adm_ons_name' in df.columns:
    df['adm_ons_name'] = df['adm_ons_name'].astype(str).str.strip().str.upper()
    df['county'] = df['adm_ons_name'].map(county_map).fillna('UNKNOWN')
    df.drop(columns=['adm_ons_name'], inplace=True)

    categorical_cols = [col for col in categorical_cols if col != 'adm_ons_name']
    categorical_cols.append('county')
else:
    print("'adm_ons_name' not found. Skipping county mapping.")

# Encode
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        # Optional decode maps
        if col == 'gender':
            df[col] = df[col].map({'M': 'Male', 'F': 'Female'}).fillna('Unknown')

        if col == 'marital_status':
            df[col] = df[col].map({
                'C': 'Civil_Partnership', 'D': 'Divorced', 'M': 'Married',
                'S': 'Single', 'W': 'Widowed', 'P': 'Declared_Partnership',
                'U': 'Unknown'
            }).fillna('Unknown')

        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

# Numeric Columns to Scale
numeric_cols_to_scale = [
    'actual_pay_over_12_months_final_salary_scheme',
    'pensionable_pay_final_salary_scheme',
    'rate_of_pensionable_pay_final_salary_scheme',
    'pensionable_service_final_salary_scheme',
    'part_time_proportion',
    'pre_88_gmp',
    'post_88_gmp',
    'accrued_2015_scheme_pension_at_valuation_date'
]

# Age columns
age_cols = ['age_at_join', 'years_until_state_pension', 'years_in_scheme']

# Fill missing
df[numeric_cols_to_scale + age_cols] = df[numeric_cols_to_scale + age_cols].fillna(0)

# and scale
scaler = StandardScaler()
df[numeric_cols_to_scale] = scaler.fit_transform(df[numeric_cols_to_scale])


#  Select original columns to keep for EDA
keep_readable = ['current_scheme', 'gender', 'marital_status', 'county']

# Build encoded column list
encoded_cols = [f"{col}_encoded" for col in categorical_cols]

# Rearrange readable + encoded pairs (for clarity)
ordered_categoricals = []
for col in keep_readable:
    if col in df.columns and f"{col}_encoded" in df.columns:
        ordered_categoricals.append(col)
        ordered_categoricals.append(f"{col}_encoded")

df_processed = df[
    numeric_cols_to_scale + age_cols + ordered_categoricals +
    [col for col in encoded_cols if col not in ordered_categoricals]
]

df_processed.columns
df_processed.info()
df_processed.describe(include='all')


# **Build Final Dataset**

raw_features_to_add = [
    'rank',
    'rank_orig',
    'rank_label',
    'rank_orig_label',
    'service_on_full_time_basis_final_salary_scheme',
    'transferred_in_service_final_salary_scheme',
    'dependant_service_final_salary',
    'actual_pensionable_earnings_over_year_police_2015_scheme',
    'annual_rate_of_pensionable_earnings_police_2015_scheme',
    'active_service_on_full_time_basis_2015_scheme',
    'x2015_scheme_club_pension_subject_to_cpi_0_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_25_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_5_percent_increases',
    'x2015_scheme_club_pension_subject_to_cpi_1_6_percent_increases',
    'x2015_scheme_club_pension_subject_to_earnings_increases',
    'x2015_rate_of_pensionable_pay_final_salary_scheme',
    'x2016_rate_of_pensionable_pay_final_salary_scheme',
    'x2017_rate_of_pensionable_pay_final_salary_scheme',
    'x2018_rate_of_pensionable_pay_final_salary_scheme',
    'x2019_rate_of_pensionable_pay_final_salary_scheme',
    'x2015_16_employee_contribution_history',
    'x2016_17_employee_contribution_history',
    'x2017_18_employee_contribution_history',
    'x2018_19_employee_contribution_history',
    'x2019_20_employee_contribution_history',
    'x2015_scheme_reckonable_service',
    'actual_pay_over_12_months_final_salary_scheme_orig',
    'pensionable_pay_final_salary_scheme_orig',
    'date_of_birth_month',
    'date_joined_scheme_month',
    'date_of_transition_to_2015_scheme_month',
    'date_attains_state_pension_age_legislation_month'
]

existing_raw_cols = [col for col in raw_features_to_add if col in df.columns]

df_final = df_processed.join(df[existing_raw_cols])

df_final.columns

missing = df_final.isnull().sum().sort_values(ascending=False)
display(missing[missing > 0])

df_final['rank_label'].value_counts(dropna=False).sort_index()

df_final.to_csv("processed_data.csv", index=False)

# Split Final Dataset By Gender

# Split using encoded gender
df_females = df_final[df_final['gender_encoded'] == 0].copy()
df_males = df_final[df_final['gender_encoded'] == 1].copy()

df_females.to_csv("females.csv", index=False)
df_males.to_csv("males.csv", index=False)